In [1]:
# This code from
# https://medium.com/data-science-in-your-pocket/multi-label-classification-for-beginners-with-codes-6b098cc76f99

In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.metrics import F1Score
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
#df.to_pickle("/content/drive/MyDrive/Colab Notebooks/df_w_embeddings_ohe.pkl")
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/df_w_embeddings_ohe.pkl")

In [4]:
# Initial train and test split.
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, valid_df = train_test_split(train_df, test_size=0.25, random_state=42) #0.25 x 0.8 = 0.2

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")
print(f"Number of rows in validation set: {len(valid_df)}")

Number of rows in training set: 10800
Number of rows in test set: 3600
Number of rows in validation set: 3600


In [5]:
df

,documentId,postures,sections,concatenated_text,embedding,ohe_postures,ohe_postures_float
0,Ib4e590e0a55f11e8a5d58a2c8dcb28b5,[On Appeal],"[{'headtext': '', 'paragraphs': ['Plaintiff Dw...",Plaintiff Dwight Watson (“Husband”) appeals f...,"[0.13452414, 0.1782438, -0.054326914, 0.307920...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Ib06ab4d056a011e98c7a8e995225dbf9,"[Appellate Review, Sentencing or Penalty Phase...","[{'headtext': '', 'paragraphs': ['After pleadi...","After pleading guilty, William Jerome Howard,...","[0.038121928, 0.32856792, -0.3703702, 0.239396...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Iaa3e3390b93111e9ba33b03ae9101fb2,"[Motion to Compel Arbitration, On Appeal]","[{'headtext': '', 'paragraphs': ['Frederick Gr...","Frederick Greene, the plaintiff below, deriva...","[0.3980108, 0.35936612, -0.32921728, 0.1975094...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,I0d4dffc381b711e280719c3f0e80bdd0,"[On Appeal, Review of Administrative Decision]","[{'headtext': '', 'paragraphs': ['Appeal from ...",Appeal from an amended judgment of the Suprem...,"[0.17980985, 0.25053117, 0.13405542, 0.2323921...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,I82c7ef10d6d111e8aec5b23c3317c9c0,[On Appeal],"[{'headtext': '', 'paragraphs': ['Order, Supre...","Order, Supreme Court, New York County (Arthur...","[0.030824184, 0.04649551, 0.020237295, -0.3026...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
17995,Ia5743cf0e4b611e99e94fcbef715f24d,[Appellate Review],"[{'headtext': '', 'paragraphs': ['¶1 On Februa...","¶1 On February 5, 2017, a jury in the Fifth J...","[0.32977995, -0.38087356, -0.35429198, 0.02168...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17996,I974c18f08f1611e998e8870e22e55653,[Objection to Proof of Claim],[{'headtext': 'ORDER OVERRULING DEBTOR'S OBJEC...,ORDER OVERRULING DEBTOR'S OBJECTION TO CLAIMS ...,"[0.16209525, -0.14097951, 0.057170607, -0.3524...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17997,Idaaa92f0886f11e998e8870e22e55653,"[Appellate Review, Trial or Guilt Phase Motion...","[{'headtext': '', 'paragraphs': ['A jury convi...",A jury convicted Antonio Avila Medrano of Con...,"[0.22042644, 0.1092553, -0.8069598, 0.28300485...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
17998,I247a8420677e11e9a072efd81f5238d6,"[Appellate Review, Jury Selection Challenge or...","[{'headtext': '', 'paragraphs': ['Defendant Ch...","Defendant Charles York Walker, Jr., appeals f...","[0.35001236, -0.10290279, -0.6039653, 0.064887...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
# This will get as far as loading the data but if you try to run classifier.fit it runs out of RAM
# It would be nice to have a baseline of some kind such as logistic regression but this isn't practical for my runtime environment.

'''
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# Generating a random multilabel dataset with 5 feature"s and 4 possible labels
#X, y = make_multilabel_classification(n_samples=1000, n_features=5, n_classes=4, n_labels=3, random_state=42)

# Splitting the dataset into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = train_df['embedding'].values
X_test = test_df['embedding'].values
y_train = train_df['ohe_postures'].values
y_test = test_df['ohe_postures'].values

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
X_train = mlb.fit_transform(X_train)
X_test = mlb.fit_transform(X_test)
y_train = mlb.fit_transform(y_train)
y_test = mlb.fit_transform(y_test)

# Creating the MultiOutput Classifier with Logistic Regression as the base estimator
classifier = MultiOutputClassifier(LogisticRegression(max_iter=1000))

# Fitting the classifier on the training data
classifier.fit(X_train, y_train)

# Making predictions on the test set
predictions = classifier.predict(X_test)

# Evaluate the model, for example, using accuracy score
accuracy = classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
'''

'\nfrom sklearn.datasets import make_multilabel_classification\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.multioutput import MultiOutputClassifier\nfrom sklearn.linear_model import LogisticRegression\n\n# Generating a random multilabel dataset with 5 feature"s and 4 possible labels\n#X, y = make_multilabel_classification(n_samples=1000, n_features=5, n_classes=4, n_labels=3, random_state=42)\n\n# Splitting the dataset into train and test sets\n#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\nX_train = train_df[\'embedding\'].values\nX_test = test_df[\'embedding\'].values\ny_train = train_df[\'ohe_postures\'].values\ny_test = test_df[\'ohe_postures\'].values\n\nfrom sklearn.preprocessing import MultiLabelBinarizer\nmlb = MultiLabelBinarizer()\nX_train = mlb.fit_transform(X_train)\nX_test = mlb.fit_transform(X_test)\ny_train = mlb.fit_transform(y_train)\ny_test = mlb.fit_transform(y_test)\n\n# Creating the MultiOutput 

In [7]:
X_train = np.vstack(train_df['embedding'].values).astype(np.float32)
X_test = np.vstack(test_df['embedding'].values).astype(np.float32)
y_train = np.vstack(train_df['ohe_postures'].values).astype(np.float32)
y_test = np.vstack(test_df['ohe_postures'].values).astype(np.float32)
X_valid = np.vstack(valid_df['embedding'].values).astype(np.float32)
y_valid = np.vstack(valid_df['ohe_postures'].values).astype(np.float32)

# Creating a multilabel neural network using TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(768,)),  # Input layer with 768 features
    tf.keras.layers.Dense(64, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(224, activation='sigmoid')  # Output layer with 224 nodes and sigmoid activation for multilabel classification
])

#Early stopping for when validation loss starts going back up again.
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=3,
                               restore_best_weights=True,
                               verbose=1)

#This actually makes the F1 worse
#model = tf.keras.Sequential([
#    tf.keras.layers.Dense(256, activation='relu', input_shape=(768,)),
#    tf.keras.layers.Dropout(0.3),  # Helps prevent overfitting
#    tf.keras.layers.Dense(128, activation='relu'),
#    tf.keras.layers.Dropout(0.3),
#    tf.keras.layers.Dense(64, activation='relu'),
#    tf.keras.layers.Dense(224, activation='sigmoid')  # Output layer
#])

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Default is 0.001, try lowering it
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[F1Score(average='weighted', threshold=None, name="f1_score", dtype=None)])

# Training the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - f1_score: 0.2603 - loss: 0.1509 - val_f1_score: 0.5003 - val_loss: 0.0153
Epoch 2/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5118 - loss: 0.0146 - val_f1_score: 0.5283 - val_loss: 0.0135
Epoch 3/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5411 - loss: 0.0128 - val_f1_score: 0.5396 - val_loss: 0.0127
Epoch 4/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5477 - loss: 0.0122 - val_f1_score: 0.5474 - val_loss: 0.0120
Epoch 5/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5623 - loss: 0.0114 - val_f1_score: 0.5558 - val_loss: 0.0114
Epoch 6/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5642 - loss: 0.0111 - val_f1_score: 0.5670 - val_loss: 0.0113
Epoch 7/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5750 - loss: 0.0106 - val_f1_score: 0.5637 - val_loss: 0.0110
Epoch 8/30
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - f1_score: 0.5763 - loss: 0.0103 - val_f1_score: 0.

In [8]:
model.evaluate(X_valid, y_valid)

113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.5783 - loss: 0.0105


[0.010243387892842293, 0.5812605619430542]